# Discussion 4

### Due Saturday October 31, 11:59:59PM



---


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import os

# helper function for the notebook in util.py
# take a look at the file!
from util import multi_table

# Review: Combining DataFrames 

#### `merge()`

* Used to combine two (or more) dataframes on the basis of **values of common columns** (indices can also be used, use `left_index=True` and/or `right_index=True`).
    * If we are joining columns on columns, the DataFrame indexes will be ignored. 
    * If we are joining indexes on indexes or indexes on a column or columns, the index will be passed on.

In [ ]:
# left dataframe
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Aaron', 'Marina', 'Justin', 'Janine', 'Ilya'],
   'subject_id':['sub1','sub2','sub4','sub6','sub5']})

# right dataframe
right = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Enrique', 'Parker', 'Erik', 'Allston', 'Betty'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})

multi_table([left, right])

* **`on`**: column or index level names to join on. 
    * These must be found in both DataFrames. 
    * If `on` is `None` and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

In [ ]:
# merge left and right tables on 'id' column
on_id = pd.merge(left,right,on='id')

# how many rows, how many columns?
multi_table([left, right, on_id])

In [ ]:
# merge left and right tables on 'id' and 'subject_id' columns
on_id_subject = pd.merge(left,right,on=['id', 'subject_id'])

# how many rows, how many columns, what are the indices?
multi_table([left, right, on_id_subject])

* **`how`**: specifies how to determine which keys are to be included in the resulting table. 
    * If a key (column name) combination does not appear in either the left or the right tables, the values in the joined table will be `np.NaN`.
    * Defaults to `inner`, joining will be performed on index. 

In [ ]:
data_a = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Manny', 'Will', 'Hunter', 'Ian', 'Eric'], 
        'last_name': ['Machado', 'Myers', 'Renfroe', 'Kinsler', 'Hosmer']}
df_a = pd.DataFrame(data_a, columns = ['subject_id', 'first_name', 'last_name'])

data_b = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Cody', 'Justin', 'Corey', 'Clayton', 'Kenley'], 
        'last_name': ['Bellinger', 'Turner', 'Seager', 'Kershaw', 'Jansen']}
df_b = pd.DataFrame(data_b, columns = ['subject_id', 'first_name', 'last_name'])

multi_table([df_a, df_b])

| Merge Method  | Description                  |
| :-------      | :---------------------------:| 
| `left`        | Use keys from left object    | 
| `right`       | Use keys from right object   | 
| `outer`       | Use union of keys            |
| `inner`       | Use intersection of keys     | 

In [ ]:
# based on the output below, what 'how' argument was passed into pd.merge?
how_list = ['outer', 'inner', 'right', 'left']

merge_method = np.random.choice(how_list)

pd.merge(df_a, df_b, on='subject_id', how=merge_method)

In [ ]:
# let's check!
merge_method

#### `concat()`

* Used to append one (or more) dataframes one below the other (or sideways, depending on whether the axis option is set to 0 or 1).
    * Useful if we have two or more data sets containing the same columns but different rows of data.
    * We can also the columns from one `Dataframe` to those of another `Dataframe`.

In [ ]:
# add 'left' below 'right'
pd.concat([right, left])

In [ ]:
# if you want to keep track of the names dataframes after concat, use 'keys'
pd.concat([right, left], keys=['right', 'left'])

In [ ]:
# add 'left' to the side of 'right'
pd.concat([right, left], axis=1)

#### `join()`

* Used to merge two dataframes on the basis of the index; instead of using `merge()` with the option `left_index=True` we can use `join()`.
    * Join operation honors the object on which it is called: `a.join(b)` $ \neq $ `b.join(a)`.

<img src="join_types.jpg">

1. **Inner Join** – default behavior, only keep rows where the merge “on” value exists in both the left and right dataframes.
2. **Left Outer** – keep every row in the left dataframe.
    * Where there are missing values of the “on” variable in the right dataframe, add `np.NaN` values in the result.
3. **Right Join** – keep every row in the right dataframe. 
    * Where there are missing values of the “on” variable in the left column, add `np.NaN` values in the result.
4. **Outer Join** – returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with `NaNs` elsewhere.

We'll start with a simple example:

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'val': [1, 2]}).set_index('key')
right = pd.DataFrame({'key': ['foo', 'bar'], 'val': [4, 5]}).set_index('key')

joined = left.join(right, lsuffix='_l', rsuffix='_r')

multi_table([left, right, joined])

Now let's try something a bit more complex:

In [ ]:
df1_data = {
    'Year' : [2014, 2014, 2014, 2014, 2014],
    'Week' : ['A', 'B', 'B', 'C', 'D'],
    'Color' : ['Red', 'Red', 'Black', 'Red', 'Green'],
    'Val' : [50, 60, 70, 10, 20]
}

df1 = pd.DataFrame(df1_data).set_index('Week')

df2_data = {
    'Year' : [2014, 2014, 2014, 2014, 2014],
    'Week' : ['A', 'B', 'C', 'C', 'D'],
    'Color' : ['Black', 'Black', 'Green', 'Red', 'Red'],
    'Score' : [30, 100, 50, 20, 40]
}

df2 = pd.DataFrame(df2_data).set_index('Week')

multi_table([df1, df2])

In [ ]:
# how many rows, how many columns?
df1.join(df2, lsuffix='_l', rsuffix = '_r')

In [ ]:
# will this be any different?
df2.join(df1, lsuffix='_l', rsuffix = '_r')

---

## Practice Problems

**Question 1**

You are given two seperate dataframes: `mlb_2017` and `mlb_2018`. Both dataframes contain statistics for the 2017 and 2018 baseball seasons respectively. Your job is two combine these two dataframes into one using the following guidelines:

* The dataframe you return should be indexed by team name (`Tm`).
* The dataframe you return should include all columns from both `mlb_2017` and `mlb_2018`.
* Use the suffixes `_2017` and `_2018` to differentiate between statistics from both seasons.

Create a function `combined_seasons` that returns, as a tuple, the following:

* The combined dataframe described above.
* The team with most homeruns (`HR`) bewteen the 2017 and 2018 seasons combined.

In [ ]:
# read in the following .txt files
mlb_2017 = pd.read_csv(os.path.join('data','mlb_2017.txt'))
mlb_2018 = pd.read_csv(os.path.join('data','mlb_2018.txt'))

multi_table([mlb_2017.head(), mlb_2018.head()])

**Question 2**

Using the same two dataframes, `mlb_2017` and `mlb_2018`, create a function `seasonal_average` that combines them and takes the mean of each column for each team. 

* The dataframe you return should be indexed by team name (`Tm`).
* Each column should contain the average value between the *2017* and *2018* seasons for the given statistic for each team.
    * For example, the `HR` column should contain the average value for `HR` for each team between the *2017* and *2018* seasons.